In [ ]:
%pip install overturemaps lonboard geopandas shapely

In [21]:
%pip install google.generativeai

Defaulting to user installation because normal site-packages is not writeable
  Using cached google_auth-2.39.0-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 16.9 MB/s eta 0:00:00
Using cached google_auth-2.39.0-py2.py3-none-any.whl (212 kB)
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   ----------- ---------------------------- 3.9/13.3 MB 20.4 MB/s eta 0:00:01
   ---------------------- ----------------- 7.6/13.3 MB 17.8 MB/s eta 0:00:01
   ------------------------------------ --- 12.1/13.3 MB 18.8 MB/s eta 0:00:01
   ---------------------------------------- 13.3/13.3 MB 18.0 MB/s eta 0:00:00
Using cached rsa-4.9.1-py3-none-any.whl (34 kB)
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ------------------------------------ --- 3.9/4.3 MB 19.5 MB/s eta 0:00:01
   


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\chaav\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import overturemaps as om
from overturemaps import core
import overturemaps
import pandas
import geopandas as gpd
from shapely import wkb
from lonboard import Map, PolygonLayer, ScatterplotLayer
import ipywidgets as widgets
import numpy as np
from IPython.display import display


In [2]:
def create_map(dataset):
    layer = ScatterplotLayer.from_geopandas(
        dataset,
        get_fill_color=[255, 0, 0],
        radius_min_pixels=5,
    )

    view_state = {
        "longitude": (bbox[0] + bbox[2]) / 2,
        "latitude": (bbox[1] + bbox[3]) / 2,
        "zoom": 8,
        "pitch": 45,
    }
    m = Map(layer, view_state=view_state)
    return m

In [ ]:
# specify bounding box
bbox = 9.0894, 45.4642, 9.1494, 45.5242

In [4]:
place_dataset = core.geodataframe("place", bbox=bbox)
print(place_dataset.shape)

(4802, 15)


In [ ]:
# Read the categories.txt file
with open('Data/categories.txt', 'r') as file:
    categories_data = file.readlines()
categories_data = categories_data[1:]  # Skip the header line
# print(np.shape(categories_data))


# Split the data by semicolons and extract the part before the semi-colon
categories_list = [entry.split(';')[0] for entry in categories_data]
print(str(categories_list))

# Print the extracted parts in a readable format
# for category in categories_list:
#     print(category)

['eat_and_drink', 'restaurant', 'afghan_restaurant', 'african_restaurant', 'ethiopian_restaurant', 'senegalese_restaurant', 'south_african_restaurant', 'moroccan_restaurant', 'nigerian_restaurant', 'american_restaurant', 'arabian_restaurant', 'belgian_restaurant', 'latin_american_restaurant', 'argentine_restaurant', 'belizean_restaurant', 'bolivian_restaurant', 'brazilian_restaurant', 'chilean_restaurant', 'colombian_restaurant', 'costa_rican_restaurant', 'cuban_restaurant', 'ecuadorian_restaurant', 'guatemalan_restaurant', 'honduran_restaurant', 'mexican_restaurant', 'nicaraguan_restaurant', 'panamanian_restaurant', 'paraguayan_restaurant', 'peruvian_restaurant', 'puerto_rican_restaurant', 'salvadoran_restaurant', 'texmex_restaurant', 'uruguayan_restaurant', 'venezuelan_restaurant', 'middle_eastern_restaurant', 'armenian_restaurant', 'azerbaijani_restaurant', 'egyptian_restaurant', 'georgian_restaurant', 'israeli_restaurant', 'kofta_restaurant', 'kurdish_restaurant', 'lebanese_restaur

In [ ]:
#visualize the map
create_map(place_dataset)

In [14]:
# Pulling out the places with no alt_categories
no_alt = {}
number = 0
for i in range(len(place_dataset.id)):
    # print(f"Place: {place_dataset.names[i]['primary']}")
    # number += 1
    if place_dataset.categories[i]['alternate'] is None:
        # Create a dictionary with the primary name as the key and the category as the value
        no_alt[place_dataset.names[i]['primary']] = place_dataset.categories[i]['primary']

# print(number)

In [ ]:
# Number of places with no alt_categories
print(len(no_alt.keys()))

# Print the places with no alt_categories
print(str(dict(list(no_alt.items()))))



570
{'Antica Chiesa di Baggio': 'church_cathedral', 'Lo Stanzino': 'community_services_non_profits', 'FitActive Bovisio Masciago': 'college_university', 'Lecafèhabitat': 'bar', 'Centro Giardinaggio': 'nursery_and_gardening', 'DRIVALIA Mobility Store': 'travel', 'ApePè': 'event_planning', 'Novità': "women's_clothing_store", 'Farmacia Zoia': 'shopping', 'Circolo Pd Fratelli Cervi - Quarto Cagnino Municipio 7 Milano': 'political_party_office', "Allarga l'arca": 'professional_services', 'Ammar Ahmed Edile Costruzioni': 'building_contractor', '14-24': 'medical_service_organizations', 'UniversitÀ Degli Studi Di Milano-Medicina E Chirurgia': 'college_university', 'Casa Pignorata Cosa Fare': 'professional_services', 'Linklabour': 'career_counseling', 'CRAL Italtel ': 'sports_and_recreation_venue', 'GameHosting': 'internet_service_provider', 'PMI Hub': 'credit_and_debt_counseling', 'Carrefour Italia S.p.A.': 'mass_media', 'Mare Milano': 'bar', '33 Beat Studio': 'music_production', 'Milano da Ve

In [ ]:
# Set up the API key
api_key = <Your Gemini API Key>

In [9]:
# Initialize the Generative AI API
import google.generativeai as genai
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.0-flash")
context = "You are a Category Suggestion Agent. Your job is to read a POI record that I provide you with and propose an alternative category for it using its name and category."+\
        f"These are all the categories in the schema:{str(categories_list)}." +\
        "I am going to provide you with all the POIs with no alternative category, I will provided the names and the primary categorys in this format: 'name': 'primary category'" +\
        f"These are the POIs: {str(dict(list(no_alt.items())))}" +\
        "Please provide me with a category that best sutes the alternative category for these POIs from the list of categories I provided you with."+\
        "The response should be in the format: 'name': 'primary category' & 'alternate category'."

In [ ]:
# Generate content using the model
response = model.generate_content(context)
print(response.text)

Here are the suggested alternative categories for the POIs you provided:

*   'Antica Chiesa di Baggio': 'church_cathedral' & 'religious_organization'
*   'Lo Stanzino': 'community_services_non_profits' & 'social_service_organizations'
*   'FitActive Bovisio Masciago': 'college_university' & 'gym'
*   'Lecafèhabitat': 'bar' & 'cafe'
*   'Centro Giardinaggio': 'nursery_and_gardening' & 'home_and_garden'
*   'DRIVALIA Mobility Store': 'travel' & 'car_rental_agency'
*   'ApePè': 'event_planning' & 'food_truck'
*   'Novità': "women's_clothing_store" & 'clothing_store'
*   'Farmacia Zoia': 'shopping' & 'pharmacy'
*   'Circolo Pd Fratelli Cervi - Quarto Cagnino Municipio 7 Milano': 'political_party_office' & 'political_organization'
*   "Allarga l'arca": 'professional_services' & 'charity_organization'
*   'Ammar Ahmed Edile Costruzioni': 'building_contractor' & 'home_service'
*   '14-24': 'medical_service_organizations' & 'health_and_medical'
*   'UniversitÀ Degli Studi Di Milano-Medicina E